# L03: NG Feasibility Study

Assumptions:
- when dispatched, plant dispatches at full capacity
- max capacity * 8760h = 438GWh
- cost of NG fixed $2.87/MMBtu

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

In [72]:
# read data
# read into pd dataframe
df = pd.read_csv("./vermont_demand_price_2022.csv")
df2 = pd.read_csv("./Data_wind_temp.csv")

# Extract data
pprice = df["Price ($/MWh)"]
load = df["Load (MW)"]
temp = df2[" temperature (F)"]

print(pprice.iloc[0])

25.56


In [129]:
# steam demand function
# where Hh is the steam demand in hour h, and Th is the outside temperature (in F) in
# hour h. For example, when the temperature is 60 F, the hospital will purchase 1 million
# btu of steam. When the temperature is 0 F, the hospital will purchase up to 7 million
# btu of steam.

hh_list = []
for h in range(8760):
    hh = max((70-temp[h])*1e5, 0)
    hh_list.append(hh)

steam_demand = pd.DataFrame(hh_list, columns=["steam demand per hour"])

In [124]:
# vars
e_ = []
for i in range(8760):
    e_.append(51 * .38) # .38 efficiency wrt electricity

# eeout = 51 MWh, assuming eout dispatch at max power always, max out of turbine
eeout = pd.DataFrame(e_, columns=["eeout"])
s_price = 15 # dollars per MMBtu
install_cost = 1000 # dollars per kW (install cost of the thing
ng_price = 2.9 # / $/mmbtu
mmbtu_to_mwh = 1 / 0.293071

In [125]:
def is_dispatched(a,b,c,d):
    if (a + b) > (c + d):
        return 1
    return 0

In [126]:
# 1
elec_rev = 0
profit_sum = 0
for i in range(8760):
    ein = eeout.iloc[i,0] / efficiency
    elec_rev = pprice.iloc[i] * eeout.iloc[i,0]
    steam_rev = s_price * min(steam_demand.iloc[i,0], ein*.4)
    
    
    ng_cost = ng_price * ein * mmbtu_to_mwh

    o_m = 6 * eeout.iloc[i,0] # 6 dollars per MWh

    d_factor = is_dispatched(elec_rev, steam_rev, ng_cost, o_m)

    profit_sum += (( elec_rev + steam_rev - ng_cost - o_m ) * d_factor)
    # print("elec_rev", elec_rev) #1 
    # print("steam_rev", steam_rev) #2
    # print("ng_cost", ng_cost) #3
    # print("o_m", o_m) #4
    # print(d_factor)
    # print(profit_sum)

print(profit_sum)

11221954.118743844


In [128]:
# compute annual cost
n = 20
r = .05
P = 4e6

rn = (1+r) ** n

A = P * r * rn / (rn -1)
print(A)

320970.3487627652
